# Re-Chunking Data

This notebook extends ideas covered in the [basic workflow](./ReChunkingData.ipynb).  This 
notebook will perfrom the same operations, but will work on the **much** larger dataset, and 
involve some parallelization using the dask scheduler. 

:::{Warning}

You should run this **only** on a cloud compute node.  We will be reading and writing _enormous_ 
amounts of data to S3 buckets. To do that over a typical network connection will saturate your 
bandwidth and take days to complete.

:::

## System Setup 

In [24]:
# Include HyTest helpers...
import sys
libDir = r'/shared/users/lib'
if libDir not in sys.path:
    sys.path.append(libDir)
# Activate logging
import logging
logging.basicConfig(level=logging.INFO, force=True)

# We're going to need a bigger boat....  8CPU / 32Gb would be ideal
import os
print(f"CPUS: {os.cpu_count()}")
import psutil
svmem = psutil.virtual_memory()
print(f"Total Virtual Memory: {svmem.total/(1024*1024*1024):.2f} Gb")

CPUS: 8
Total Virtual Memory: 30.58 Gb


## Plumb Data Source
List available datasets at the National Water Model Reanalysis Version 2.1. 
The dataset is part of the AWS Open Data Program

In [23]:
import fsspec
fs = fsspec.filesystem('s3', anon=True)
fs.ls('s3://noaa-nwm-retrospective-2-1-zarr-pds/')

['noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr',
 'noaa-nwm-retrospective-2-1-zarr-pds/gwout.zarr',
 'noaa-nwm-retrospective-2-1-zarr-pds/index.html',
 'noaa-nwm-retrospective-2-1-zarr-pds/lakeout.zarr',
 'noaa-nwm-retrospective-2-1-zarr-pds/ldasout.zarr',
 'noaa-nwm-retrospective-2-1-zarr-pds/precip.zarr',
 'noaa-nwm-retrospective-2-1-zarr-pds/rtout.zarr']

Load the `chrtout` zarr data

In [2]:
# Load chrtout
import xarray as xr
fileHandle = fs.get_mapper('noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr')
ds = xr.open_zarr(fileHandle, consolidated=True)

## Spin up Dask Cluster
This operation will be able to work in parallel.  Spin up a dask cluster on 
the cloud hardware to schedule the various workers.  Note that this cluster
is configured with a specific user **profile** with permissions to write to
our eventual output location. 

In [6]:
import ebdpy as ebd
os.environ['AWS_PROFILE'] = 'nhgf-development'
client,cluster = ebd.start_dask_cluster(
    profile=os.environ['AWS_PROFILE'],
    worker_max=30,
    region='us-west-2', 
    use_existing_cluster=True,
    adaptive_scaling=False, 
    wait_for_cluster=False, 
    propagate_env=True)
client.dashboard_link

Region: us-west-2
Existing Dask clusters:
Cluster Index c_idx: 0 / Name: dev.c5ca28826e1e425faaa77f3da798e273 ClusterStatus.RUNNING
Using existing cluster [0].
Setting Fixed Scaling workers=30
Reconnect client to clear cache
client.dashboard_link (for new browser tab/window or dashboard searchbar in Jupyterhub):
https://jupyter.qhub.esipfed.org/gateway/clusters/dev.c5ca28826e1e425faaa77f3da798e273/status
Propagating environment variables to workers
Using environment: users/pangeo


'https://jupyter.qhub.esipfed.org/gateway/clusters/dev.c5ca28826e1e425faaa77f3da798e273/status'

## Read Sample Data

In [8]:
import dask
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    smplData = ds.where(ds.gage_id != ''.rjust(15).encode(), drop=True) # subset to only those features with a valid gage_id
    smplData.drop('crs') # Not needed/wanted for this analysis
smplData

<xarray.Dataset>
Dimensions:     (feature_id: 7994, time: 367439)
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(7994,), meta=np.ndarray>
  * feature_id  (feature_id) int32 3109 3923 12932 ... 1170023539 1180000535
    gage_id     (feature_id) |S15 dask.array<chunksize=(7994,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(7994,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(7994,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(7994,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:0...
Data variables:
    crs         (feature_id) object dask.array<chunksize=(7994,), meta=np.ndarray>
    streamflow  (time, feature_id) float64 dask.array<chunksize=(672, 36), meta=np.ndarray>
    velocity    (time, feature_id) float64 dask.array<chunksize=(672, 36), meta=np.ndarray>
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

## Re-Chunk Plan
We will configure a new chunking plan which will favor time-series analysis. 
Using the dimensions of the data: 
* 367439 time steps
* 7994 feature IDs

We can write the new plan as: 

In [9]:
# The new chunking plan:
chunk_plan = {
    'streamflow': {'time': 367439, 'feature_id': 1}, # all time records in one chunk for each feature_id
    'velocity': {'time': 367439, 'feature_id': 1},
    'elevation': (7994,),
    'gage_id': (7994,),
    'latitude': (7994,),
    'longitude': (7994,),    
    'order': (7994,),    
    'time': (367439,), # all time coordinates in one chunk
    'feature_id': (7994,) # all feature_id coordinates in one chunk
}


In [10]:
# Manually reset the chunking metadata in prep for re-chunking
smplData = smplData.chunk(chunks={'feature_id':1, 'time': 367439})
for x in smplData.variables:
    smplData[x].encoding['chunks'] = None

## Set up output location

With this plan, we can ask `rechunker` to re-write the data using the prescribed chunking pattern.

Unlike with the smaller dataset, we need to write this very large dataset to an object store in the datacenter: an S3 'bucket'.  So we need to set that up so that `rechunker` will have a suitable place to write data. This new data will be a complete copy of the original, just re-organized a bit. 

In [13]:
# with anon=False, we force the use of the environment variable 'AWS_PROFILE', set above.
fsw = fsspec.filesystem('s3', anon=False, default_fill_cache=False, skip_instance_cache=True)
workspace = 's3://nhgf-development/workspace/'
testDir = workspace + "testing/"
myDir = testDir + 'ReChunkTutorial/'
fsw.ls(testDir)

INFO:aiobotocore.credentials:Found credentials in environment variables.


['nhgf-development/workspace/testing/conus404_subset.zarr',
 'nhgf-development/workspace/testing/coords.zarr',
 'nhgf-development/workspace/testing/coords2.zarr',
 'nhgf-development/workspace/testing/foo',
 'nhgf-development/workspace/testing/foobar.zarr',
 'nhgf-development/workspace/testing/tmp.zarr']

In [14]:
fsw.mkdir(myDir)

In [25]:
for f in ['rechunked.zarr', 'staging.zarr']:
    if fsw.exists(myDir + f):
        fsw.rm(myDir + f, recursive=True)
staging = fsw.get_mapper(myDir + 'staging.zarr')
outfile = fsw.get_mapper(myDir + 'rechunked.zarr')

## Ready to rechunk

In [19]:
import rechunker
## Recall that merely invoking rechunker does not do any work... just sorts out 
## the rechunking plan and writes metadata.
result = rechunker.rechunk(
    smplData,
    chunk_plan,
    "2GB",
    outfile, 
    temp_store=staging 
)

/home/conda/users/76634b8b76503b686ed55f14ef74288ada4ffb7c19282b669f7edfb7fc8aed9d-20220909-192621-731199-163-pangeo/lib/python3.9/site-packages/xarray/conventions.py:205: SerializationWarning: variable None has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(


In [20]:
from dask.distributed import progress, performance_report

with performance_report(filename="dask-report.html"):
    r = result.execute(retries=10)  # <<<<<  execute actually runs the rechunker plan.

KeyboardInterrupt: 

In [22]:
import zarr
_ = zarr.consolidate_metadata(outfile)

## Results
Let's read in the resulting re-chunked dataset to see how it looks:

In [ ]:
reChunkedData = xr.open_zarr(outfile)
reChunkedData

### Comparison


In [ ]:
## Before:
sampleData['streamflow'].sel(feature_id=1343034)
# Note: three chunks needed to service a single feature_id


In [ ]:
## After:
reChunkedData['streamflow'].sel(feature_id=1343034) 
# All data for the specified feature_id is in a single chunk


In [27]:
from HyTest.helpers import stop_running_clusters
stop_running_clusters()

INFO:root:Found 0 runnning dask clusters.
